In [ ]:
import os
# print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.
import itertools
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import os
# print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.
import itertools
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import pandas  as pd
import numpy as np
import os
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB,BernoulliNB
nb = MultinomialNB()
from sklearn.preprocessing import LabelEncoder, OneHotEncoder 
from sklearn.utils import class_weight
from sklearn.svm import SVC
from scipy.sparse import hstack, vstack
from nltk.corpus import stopwords  
#nltk.download('stopwords') 
import pickle
import datetime
from dateutil import parser
import sys
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import f1_score ,accuracy_score,confusion_matrix
import warnings
warnings.filterwarnings("ignore")

In [ ]:
data = pd.read_excel("C://Users//Abhishek_Prasar//Documents//My Received Files//train_29_march_preprocessed.xlsx")
data.head()

In [ ]:
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()
data['Final_Category_Encoded'] = lb.fit_transform(data['Reason_for_Cancellation_COE'])
data.head()


In [ ]:

#Preprocessing Training Set
data["NOTES_COE"] = data["NOTES_COE"].astype(str)
data["NOTES_COE"] = data["NOTES_COE"].str.upper()
data["NOTES_COE"] = [x.replace(':', ' ') for x in data["NOTES_COE"]]
#Removing Punctuations
data["NOTES_COE"]  = data["NOTES_COE"] .str.replace('[^\w\s]','')
#from nltk.corpus import stopwords
#stop = stopwords.words('english')
#data["NOTES_COE"] = data["NOTES_COE"].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
#Rare Word Removal
freq = pd.Series(' '.join(data["NOTES_COE"]).split()).value_counts()[-20:]

freq = list(freq.index)
data["NOTES_COE"] = data["NOTES_COE"].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
data["NOTES_COE"]

#Lemmatization
from textblob import Word
data["NOTES_COE"] = data["NOTES_COE"].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
data["NOTES_COE"].head()


In [ ]:
valid = pd.read_excel("C:\\Users\\Abhishek_Prasar\\Documents\\My Received Files\\test_29_march_preprocessed.xlsx")
valid[['COE_NOTES','Reason_for_Cancellation_ENT']].head()

In [ ]:
data['Final_Category_Encoded1'] = lb.fit_transform(data['Reason_for_Cancellation_COE'])
valid['Final_Category_Encoded1'] = lb.transform(valid['Reason_for_Cancellation_ENT'])

In [ ]:
valid["NOTES_COE"] = valid["NOTES_COE"].str.upper()
valid["NOTES_COE"] = [x.replace(':', ' ') for x in valid["NOTES_COE"]]
#Removing Punctuations
valid["NOTES_COE"]  = valid["NOTES_COE"] .str.replace('[^\w\s]','')
#from nltk.corpus import stopwords
#stop = stopwords.words('english')
#valid["NOTES_COE"] = valid["NOTES_COE"].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
#Rare Word Removal
freq = pd.Series(' '.join(valid["NOTES_COE"]).split()).value_counts()[-20:]

freq = list(freq.index)
valid["NOTES_COE"] = valid["NOTES_COE"].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
valid["NOTES_COE"].head()

#Lemmatization
from textblob import Word
valid["NOTES_COE"] = valid["NOTES_COE"].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
valid["NOTES_COE"].head()

In [ ]:
from sklearn.metrics import confusion_matrix

from tensorflow import keras
layers = keras.layers
models = keras.models

In [ ]:
X_train = data["NOTES_COE"]


In [ ]:
X_test = valid["NOTES_COE"]

In [ ]:
max_words = 1000
tokenize = keras.preprocessing.text.Tokenizer(num_words=max_words,char_level=False)


In [ ]:
tokenize.fit_on_texts(X_train) # fit tokenizer to our training text data
tokenize.fit_on_texts(X_test)
x_train = tokenize.texts_to_matrix(X_train)
x_test = tokenize.texts_to_matrix(X_test)

In [ ]:
y_train = data["Final_Category_Encoded1"]
y_test = valid["Final_Category_Encoded1"]

In [ ]:
# Converts the labels to a one-hot representation
num_classes = np.max(y_train) + 1
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [ ]:
# Inspect the dimenstions of our training and test data (this is helpful to debug)
print('X_train shape:', x_train.shape)
print('X_test shape:', x_test.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

In [ ]:
# Note: The data processed for this output had the max text length set to 400.
# for batch_size in range(10,31,10):
#   for epochs in range(3,15,5):
#     for drop_ratio in np.linspace(0.1, 0.5, 3):
#       run_experiment(batch_size, epochs, drop_ratio)

In [ ]:
#Traing Model
batch_size = 50
epochs = 8
drop_ratio = 0.5

In [ ]:
# Build the model
model = models.Sequential()
model.add(layers.Dense(512, input_shape=(max_words,)))
model.add(layers.Activation('relu'))
# model.add(layers.Dropout(drop_ratio))
model.add(layers.Dense(num_classes))
model.add(layers.Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:

# The validation_split param tells Keras what % of our training data should be used in the validation set
# You can see the validation loss decreasing slowly when you run this
# Because val_loss is no longer decreasing we stop training to prevent overfitting
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

In [ ]:
# Evaluate the accuracy of our trained model
score = model.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
def run_experiment(batch_size, epochs, drop_ratio):
  print('batch size: {}, epochs: {}, drop_ratio: {}'.format(
      batch_size, epochs, drop_ratio))
  model = models.Sequential()
  model.add(layers.Dense(512, input_shape=(max_words,)))
  model.add(layers.Activation('sigmoid'))
  model.add(layers.Dropout(drop_ratio))
  model.add(layers.Dense(num_classes))
  model.add(layers.Activation('softmax'))

  model.compile(loss='categorical_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])
  history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=0,
                    validation_split=0.1)
  score = model.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=0)
  print('\tTest loss:', score[0])
  print('\tTest accuracy:', score[1])

In [ ]:
batch_size = 32
epochs = 20
drop_ratio = .5
run_experiment(batch_size, epochs, drop_ratio)

In [ ]:
y_softmax = model.predict(x_test)

y_test_1d = []
y_pred_1d = []

for i in range(len(y_test)):
    probs = y_test[i]
    index_arr = np.nonzero(probs)
    one_hot_index = index_arr[0].item(0)
    y_test_1d.append(one_hot_index)

for i in range(0, len(y_softmax)):
    probs = y_softmax[i]
    predicted_index = np.argmax(probs)
    y_pred_1d.append(predicted_index)

In [ ]:
valid["predicted"]= y_pred_1d

In [ ]:
valid.to_excel("D://final_data//output_with_deep_learning.xlsx")